In [2]:
!pip install optuna xgboost scikit-learn pandas numpy matplotlib joblib -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 11.1 MB/s eta 0:00:00


In [5]:
from google.colab import drive
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")


In [6]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/GSBT Pertemuan 2-4/UCI_Credit_Card.csv'
df = pd.read_csv(path)
df.head()

Mounted at /content/drive


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [8]:
X = df.drop(columns=['ID', 'default.payment.next.month'])
y = df['default.payment.next.month']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'auc'
    }

    model = XGBClassifier(**param)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='roc_auc', n_jobs=-1)
    return np.mean(scores)

In [12]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())

[I 2025-10-19 14:20:08,678] A new study created in memory with name: no-name-8d578bbd-b35c-41ae-b377-3559a566528c


In [13]:
study.optimize(objective, n_trials=30)

[I 2025-10-19 14:20:20,932] Trial 0 finished with value: 0.7729454627518331 and parameters: {'n_estimators': 403, 'max_depth': 3, 'learning_rate': 0.23146328171054606, 'subsample': 0.7341705776135583, 'colsample_bytree': 0.5908145434242922, 'gamma': 0.7684562444750154, 'reg_alpha': 2.3113517424578207, 'reg_lambda': 4.24405764406764, 'min_child_weight': 3}. Best is trial 0 with value: 0.7729454627518331.
[I 2025-10-19 14:20:23,468] Trial 1 finished with value: 0.7718352010995695 and parameters: {'n_estimators': 113, 'max_depth': 5, 'learning_rate': 0.264152275448141, 'subsample': 0.6768646152059284, 'colsample_bytree': 0.7711884132553279, 'gamma': 0.05909470527335425, 'reg_alpha': 3.3525842896970417, 'reg_lambda': 1.2694977678392498, 'min_child_weight': 7}. Best is trial 0 with value: 0.7729454627518331.
[I 2025-10-19 14:20:31,141] Trial 2 finished with value: 0.7587086869684283 and parameters: {'n_estimators': 492, 'max_depth': 9, 'learning_rate': 0.2533573684816737, 'subsample': 0.896

In [14]:
print("Best ROC-AUC score:", study.best_value)
print("Best hyperparameters:", study.best_params)

Best ROC-AUC score: 0.786835195109085
Best hyperparameters: {'n_estimators': 411, 'max_depth': 7, 'learning_rate': 0.014783716412505575, 'subsample': 0.5745620679691358, 'colsample_bytree': 0.8659651545780391, 'gamma': 2.29668878922058, 'reg_alpha': 4.990885421117728, 'reg_lambda': 3.31213774756388, 'min_child_weight': 2}


In [15]:
best_model = XGBClassifier(**study.best_params)
best_model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8659651545780391, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=2.29668878922058, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.014783716412505575,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=411, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
y_pred_prob = best_model.predict_proba(X_test_scaled)[:, 1]
test_auc = roc_auc_score(y_test, y_pred_prob)
print("Test ROC-AUC:", test_auc)

Test ROC-AUC: 0.7801424141087887


In [18]:
import joblib
joblib.dump(study, "optuna_study_creditcard.pkl")

['optuna_study_creditcard.pkl']

In [19]:
optuna.visualization.plot_optimization_history(study)